


![Descripción de la imagen](https://drive.google.com/uc?export=download&id=1LNMJ1zjBZkWJeZwJO38EJzfOWBHt4u4q)


## **Clase 4 - GraphRAG**



# Objetivos de hoy

**Objetivo Alumno:** Entender las limitaciones del RAG basado en texto y aprender a construir una base de conocimiento en forma de grafo para permitir un razonamiento complejo.

**Objetivo Proyecto:** Crear un Knowledge Graph inicial que modele las relaciones entre productos, proveedores y pedidos.





#Material de Clase
Repositorio de Github

https://github.com/tribu-ia/don-confiado


**Este mismo colab**

[https://colab.research.google.com/drive/1c5H7VbJq47fAt_diSrkIFCP9bpTgtq8](https://colab.research.google.com/drive/1c5H7VbJq47fAt_diSrkIFCP9bpTgtq8)


URL Corta del Colab: [https://tinyurl.com/waftn9z9](https://tinyurl.com/waftn9z9)


**Colab Clase Anterior**:
https://colab.research.google.com/drive/1T15cJEjegMskuDccZoYTDbYWMiO2x1dA





## **Cómo se correlacionan RAG y LLM?**


En la clase pasada vimos como utilizar RAG para traer pedazos de texto (o alguna otra fuente de "conocimiento") desde una base de datos vectorial.


¿Cómo hacemos que ese texto o conocimiento sea usado por LLM para dar una respuesta?


Recuerdan que habíamos explicado que los LLM entre sus características tenían la capacidad de **parafrasear** y **"razonar"**.

Utilizaremos esa caracteristica para enriquecer el **Prompt** inyectando ese texto de RAG en cada llamado.

![Imagen desde Drive](https://drive.google.com/uc?export=view&id=133_lipxQacOQrtF66KP0xL3wOJ2IqQJG)





## Procesos Tradicional de RAG

![Imagen desde Drive](https://drive.google.com/uc?export=view&id=1IkMx3iBMzSpHlbFcwoFNA0sGHwQcAiEB)





#Problemas del RAG Tradicional

- Los chunks no guardan relaciones entre ellos
- Con los chunks no se puede obtener una visión global
- Los chunks traidos podrían no traer la respuesta


## Cómo lo soluciona GraphRAG

GraphRAG (Graph-based Retrieval-Augmented Generation) introduce una capa de razonamiento estructurado mediante un grafo de conocimiento (Knowledge Graph), combinando lo mejor del RAG con lo mejor de la representación simbólica.

1. **Estructura semántica explícita**

Extrae entidades (nodos) y relaciones (aristas) del corpus.

Cada nodo tiene embeddings y conexiones explícitas.

Esto permite búsquedas no solo por similitud, sino también por estructura (quién se relaciona con quién y cómo).

2. **Recuperación basada en contexto conectado**

Cuando se hace una consulta, GraphRAG puede expandir el grafo localmente para incluir entidades relacionadas, mejorando la relevancia contextual.

Así evita perder información que estaba dispersa en diferentes chunks.

3. **Razonamiento multihop**

Puede seguir relaciones en varios pasos (por ejemplo: A → B → C) para responder preguntas complejas que implican inferencia.

Esto es imposible para RAG tradicional sin cadenas de prompts complicadas.

4. **Menor redundancia, mayor precisión**

La búsqueda sobre el grafo prioriza nodos relevantes y sus relaciones, reduciendo la repetición de textos similares.

5. **Mayor interpretabilidad**

Las respuestas pueden venir acompañadas del subgrafo usado para inferir la respuesta, mostrando de manera transparente las conexiones que sustentan la generación.

6. **Mejor rendimiento en dominios especializados**

Ideal para datos empresariales, científicos o legales donde las relaciones (por ejemplo, entre proyectos, contratos, o moléculas) son más importantes que el texto literal.

# Agenda
* Setup y llaves
* Crear un Grafo
* neo4j
* Grafo + RAG + LLM







# *NO MAS TEORIA. VAMOS AL CODIGO*

In [ ]:
print("Instalando dotenv")
!pip install --quiet python-dotenv

print("Instalando openai")
!pip install --quiet openai

#print("Instalando genai")
#!pip install --quiet google-genai
print("Instalando langchain langchain_google_genai langchain_openai langchain-neo4j langchain-experimental neo4j")
!pip install --quiet langchain langchain_google_genai langchain_openai langchain-neo4j langchain-experimental neo4j
print("INSTALO DE TODO. Langchain Langchain experimental neo4j Gemini")

## Cargamos las llaves 🔑


Ahora vamos a poner las llaves o (API_KEYS) de cada plataforma


Recuerden crear los API KEYS de Google Gemini y OpenAI

Google Gemini:  https://aistudio.google.com/api-keys

OpenAI: https://platform.openai.com/api-keys


Te las pedirá o las puedes guardar para usos futuros.

![image.png](https://drive.google.com/uc?export=download&id=1D3_pfko0lAequCnGSqfQLM-q24KA2LAj)


In [ ]:
import os
from google.colab import userdata
from dotenv import load_dotenv

load_dotenv()


def assign_key(userdata,key_name:str, input_message:str, hint:str = None):
  try:
    from google.colab import userdata
    IN_COLAB = True
  except ImportError:
    IN_COLAB = False

  if IN_COLAB:
    try:
      key = userdata.get(key_name)
      if key:
        os.environ[key_name] = key
        return os.environ[key_name]
    except:
      return os.environ[key_name]

  if hint:
    print(hint)
  key = input(input_message)
  os.environ[key_name] = key
  return os.environ[key_name]



gemini_api_key = assign_key(userdata, "GEMINI_API_KEY", "Por favor, ingrese su API KEY de Gemini: ","Para obtener Gemini API Key https://aistudio.google.com/")
openai_api_key = assign_key(userdata, "OPENAI_API_KEY", "Por favor, ingrese su API KEY de OPENAI: " )
NEO4J_URI = assign_key(userdata, "NEO4J_URI", "Por favor, ingrese su URI de NEO4J: " )
NEO4J_USERNAME  = assign_key(userdata, "NEO4J_USERNAME", "Por favor, ingrese su USERNAME de NEO4J: " )
NEO4J_PASSWORD  = assign_key(userdata, "NEO4J_PASSWORD", "Por favor, ingrese su PASSWORD de NEO4J: ")
NEO4J_DATABASE = assign_key(userdata, "NEO4J_DATABASE", "Por favor, ingrese su DATABASE de NEO4J: ")


print("Ya cargaste las llaves en las variables de entorno")
print("Puedes pasar a probar los APIs")


#Definimos algunas funciones útiles para procesar archivos


Como necesitamos procesar archivos vamos a crear unas funciones que nos ayudarán a descargarlas desde Internet y extraer sus bytes

In [ ]:
import requests
import os


# Crearemos algunas funciones útiles para manipular las imágenes.


# Necesitamos cargar los bytes de una imágen en un archivo.
def load_image_bytes(path: str) -> bytes:
    with open(path, "rb") as f:
        return f.read()


#Descargamos la imagen y la guardamos localmente
def descargar_archivo(url: str, destino: str):
    """
    Descarga un archivo desde una URL (por ejemplo, de Google Drive o Imgur)
    y la guarda en la ruta especificada.

    Parámetros:
        url (str): URL directa o de descarga del archivo.
        destino (str): Ruta completa donde guardar el archivo, incluyendo el nombre del archivo.
    """
    print(f"📥 Descargando archivo desde: {url}")

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(destino, "wb") as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print(f"✅ Archivo descargado correctamente en: {destino}")
    else:
        raise Exception(f"❌ Error al descargar el archivo: {response.status_code}")

    # Verificar que el archivo existe y tiene contenido
    if not os.path.exists(destino) or os.path.getsize(destino) == 0:
        raise FileNotFoundError("❌ No se encontró el archivo o está vacio")
    else:
        print(f"📏 Tamaño del archivo: {os.path.getsize(destino)} bytes")

# Ejemplo de uso:
# descargar archivo("https://drive.google.com/uc?export=download&id=1PMXWYR_hekobCV0jSrFEz5TpIjriU6jM", "/content/imagen_imgur.png")

def descargar_archivo_a_variable(url:str, filename:str) -> str:
  descargar_archivo(url, filename)
  with open(filename, "r", encoding="utf-8") as f:
    return f.read()





## Creamos los Modelos (LLM) enLangchain

![image.png](https://drive.google.com/uc?export=download&id=1UbD5tXczZlVJ3xWH6L4eDk4fn-XkfI3V)






Se repiten los casos de clases pasadas para crear los clientes de LLM

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage , AIMessage

openai_model = init_chat_model("gpt-4.1-mini",      model_provider="openai",        api_key=openai_api_key)
gemini_model = init_chat_model("gemini-2.0-flash",  model_provider="google_genai",   api_key=gemini_api_key)

print("LLMs cargados")



## Vamos a crear un Grafo de Don Quijote




In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

#https://drive.google.com/file/d/1V7lIIJE_rv5XTXfTkvR1Qm82tZeSEKZa/view?usp=drive_link
#https://drive.google.com/file/d/1V7lIIJE_rv5XTXfTkvR1Qm82tZeSEKZa/view?usp=drive_link


text = descargar_archivo_a_variable("https://drive.google.com/uc?export=download&id=1V7lIIJE_rv5XTXfTkvR1Qm82tZeSEKZa", "/content/don-quijote-short.txt")


llm_instruction = """
en el siguiente texto, centrate en buscar relaciones y entidades, como lugares,
eventos significativos en los que participan los personajes, tipo de relacion
entre los personajes, familiaridad entre, propiedades que tienen, gustos y personalizades,
frases o poemas
"""


llm_transformer = LLMGraphTransformer(
    llm=gemini_model,
    #additional_instructions=(llm_instruction)
     allowed_nodes=["Persona Relevante","Lugar","Batalla","Aventura",  "Animal", "Objeto", ],
     allowed_relationships=["CONOCIDO", "AMA", "PELEA", "IMITA",  "FAMILIAR", "AMIGO","ESTUVO_EN", 'OCURRIO_EN', "DUEÑO_DE", "TRABAJA_PARA", "EMPLEADO_DE", "PARTICIPO", "MISMA_PERSONA"],
)


documents = [Document(page_content=text)]

print("Empieza a analizar el documento")
graph_documents = await llm_transformer.aconvert_to_graph_documents(documents)
print("Nodes")
for node in graph_documents[0].nodes:
    print(node)


print("Relationships")
for relationship in graph_documents[0].relationships:
  print(relationship)


# Una gráfica muy bonita del Grafo

¡Qué bonito! Podemos dibujar las relaciones.


In [ ]:
import asyncio
import networkx as nx
import matplotlib.pyplot as plt

graph = graph_documents[0]

# Build a NetworkX graph
G = nx.DiGraph()
for node in graph.nodes:
    G.add_node(node.id, label=node.type)
for rel in graph.relationships:
    G.add_edge(rel.source.id, rel.target.id, label=rel.type)

# Draw it
pos = nx.spring_layout(G, seed=44, k=1.5, iterations=400)
plt.figure(figsize=(16, 12))
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=1500, edge_color='gray', font_size=6)
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.title("Don quijote Knowledge Graph", fontsize=14)
plt.show()

## Problemas de este Grafo

Aunque el grafo es muy bonito y bastante impresionante no es muy útil porque debemos conocer al detalle lo que hay en cada nodo para hacer una consulta.


Necesitamos la mágia de los embeddings para poder ubicar un nodo.

Para esto usaremos **NEO4J** como base de datos de Grafos.

Las credenciales debieron ser asignadas en los primeros nodos.


In [ ]:
!pip install --quiet neo4j neo4j-graphrag langchain-neo4j langchain-experimental

#VALIDEMOS SI TENEMOS LA URL DE NEO4J



print("Debemos haber asignado NEO4J_URI ",os.environ["NEO4J_URI"])



## **===============================================**
##                    RAG   WITH EMBEDDINGS
## **===============================================**

In [ ]:
import neo4j


neo_auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])

neo4j_driver = neo4j.GraphDatabase.driver(os.environ["NEO4J_URI"],auth=neo_auth)


In [ ]:

#HARRY POTTER

##text = descargar_archivo_a_variable("https://drive.google.com/uc?export=download&id=1KFENc5Biu0LufLWx1BwhEbKfOmQ3uFFJ","/content/harry_potter.txt")
##print(text)



## MUNDIAL SUB-20   https://drive.google.com/file/d/1zuS1YzxHGqw8vqbf-_A53xXsnYAC2uGb/view?usp=drive_link
text = descargar_archivo_a_variable("https://drive.google.com/uc?export=download&id=1zuS1YzxHGqw8vqbf-_A53xXsnYAC2uGb","/content/sub20.txt")
print(text[:200])
print(text[:-200])


In [ ]:
### Limpiamos la BD
################################################################
#                          LIMIPIEZA BD
################################################################


delete_query = """
MATCH (n)
DETACH DELETE n
"""
print("Execute ",delete_query)
neo4j_driver.execute_query(delete_query)

In [ ]:
import neo4j
from neo4j_graphrag.llm import OpenAILLM as LLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings as Embeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.generation.graphrag import GraphRAG
from neo4j_graphrag.experimental.components.text_splitters.fixed_size_splitter import FixedSizeSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from neo4j_graphrag.experimental.components.text_splitters.langchain import LangChainTextSplitterAdapter



ex_llm=LLM(
   model_name="gpt-4o-mini",
   model_params={
       "response_format": {"type": "json_object"},
       "temperature": 0
   })

embedder = Embeddings()

llm_graph_instruction = """
Eres experto en extraer grafos de conocimiento a partir de textos sobre el Mundial Sub‑20 de Fútbol. El texto puede contener alineaciones, formaciones, sedes, árbitros, grupos/fases y resúmenes de partidos (goles, asistencias, tarjetas, sustituciones).

Objetivo:
- Identifica ENTIDADES (nodos) y RELACIONES (aristas) relevantes para el torneo.
- Usa exclusivamente los LABELS y RELATION TYPES permitidos listados abajo.
- Devuelve un JSON válido siguiendo estrictamente el esquema indicado.

LABELS permitidos (nodos):
- Torneo, Partido, Equipo, Seleccion, Club, Jugador, Entrenador, Arbitro,
  Estadio, Ciudad, Pais, Grupo, Fase, Premio,
  EventoGol, EventoTarjeta, EventoSustitucion

RELATION TYPES permitidos (dirigidos):
- PARTICIPA_EN (Equipo -> Torneo)
- JUGADO_EN (Partido -> Torneo | Fase | Grupo)
- LOCAL_EN (Partido -> Equipo)     // equipo local
- VISITANTE_EN (Partido -> Equipo) // equipo visitante
- SE_JUEGA_EN (Partido -> Estadio)
- UBICADO_EN (Estadio -> Ciudad), UBICADO_EN (Ciudad -> Pais)
- DIRIGE (Entrenador -> Equipo)
- JUEGA_PARA (Jugador -> Seleccion | Equipo)
- PERTENECE_A (Jugador -> Club)
- NACIONALIDAD (Jugador|Entrenador|Arbitro -> Pais)
- ARBITRA (Arbitro -> Partido)
- TITULAR_EN (Jugador -> Partido)
- SUPLENTE_EN (Jugador -> Partido)
- CAPITAN_EN (Jugador -> Partido)
- EVENTO_DE (EventoGol|EventoTarjeta|EventoSustitucion -> Partido)
- AUTOR (Jugador -> EventoGol)
- ASISTE (Jugador -> EventoGol)
- TARJETA_A (Jugador -> EventoTarjeta)
- SALE_EN (Jugador -> EventoSustitucion)
- ENTRA_EN (Jugador -> EventoSustitucion)
- GANADOR_DE (Equipo -> Partido)
- PERDEDOR_DE (Equipo -> Partido)
- EMPATE_EN (Partido -> Torneo) // usar si el partido terminó en empate
- GANA_PREMIO (Jugador|Entrenador|Equipo -> Premio)
- MIEMBRO_DE (Equipo -> Grupo)
- AVANZA_A (Equipo -> Fase)

Return result as JSON using the following format:
{{"nodes": [ {{"id": "0", "label": "the type of entity", "properties": {{"name": "name of entity" }} }}],
  "relationships": [{{"type": "TYPE_OF_RELATIONSHIP", "start_node_id": "0", "end_node_id": "1", "properties": {{"details": "Description of the relationship"}} }}] }}


Propiedades mínimas sugeridas:
- Torneo: name, year
- Partido: date (YYYY-MM-DD), time (HH:MM, opcional), fase (opcional), grupo (opcional), score_local (int, opcional), score_visitante (int, opcional)
- Equipo/Seleccion: name
- Club: name, pais (opcional)
- Jugador: name, dorsal (int, opcional), posicion (opcional)
- Entrenador/Arbitro: name
- Estadio: name, capacidad (int, opcional)
- Ciudad/Pais/Grupo/Fase/Premio: name
- EventoGol: minuto (int), tipo ("juego", "penal", "propia", opcional)
- EventoTarjeta: minuto (int), tipo ("amarilla", "roja")
- EventoSustitucion: minuto (int)

Reglas:
- No inventes entidades o datos no presentes en el texto.
- Reutiliza el mismo nodo si la entidad ya apareció (evita duplicados).
- Usa mayúsculas sostenidas para RELATION TYPES tal como se listan.
- Asegúrate de que cada id de nodo sea único y que las referencias en relaciones existan.
- Normaliza nombres: respeta tildes, usa capitalización natural (e.g., "Colombia", "España").
- Minutos y goles deben ser enteros si están presentes.

Texto de entrada:
{{text}}
"""
original_prompt="""
You are a top-tier algorithm designed for extracting
information in structured formats to build a knowledge graph.

Extract the entities (nodes) and specify their type from the following text.
Also extract the relationships between these nodes.

Return result as JSON using the following format:
{{"nodes": [ {{"id": "0", "label": "Person", "properties": {{"name": "John"}} }}],
"relationships": [{{"type": "KNOWS", "start_node_id": "0", "end_node_id": "1", "properties": {{"since": "2024-08-01"}} }}] }}

Use only the following node and relationship types (if provided):
{schema}

Assign a unique ID (string) to each node, and reuse it to define relationships.
Do respect the source and target node types for relationship and
the relationship direction.

Make sure you adhere to the following rules to produce valid JSON objects:
•⁠  ⁠Do not return any additional information other than the JSON in it.
•⁠  ⁠Omit any backticks around the JSON - simply output the JSON on its own.
•⁠  ⁠The JSON object must not wrapped into a list - it is its own JSON object.
•⁠  ⁠Property names must be enclosed in double quotes

Examples:
{examples}

Input text:

{text}
"""

# Relationship types (directed)
rel_types = [
    "PARTICIPA_EN", "JUGADO_EN", "MIEMBRO_DE", "AVANZA_A",
    "SE_JUEGA_EN", "UBICADO_EN",
    "DIRIGE", "JUEGA_PARA", "PERTENECE_A", "NACIONALIDAD",
    "LOCAL_EN", "VISITANTE_EN", "ARBITRA",
    "TITULAR_EN", "SUPLENTE_EN", "CAPITAN_EN",
    "EVENTO_DE", "AUTOR", "ASISTE", "TARJETA_A",
    "SALE_EN", "ENTRA_EN",
    "GANADOR_DE", "PERDEDOR_DE", "EMPATE_EN", "GANA_PREMIO"
]

basic_node_labels = [
    "Torneo", "Partido", "Equipo", "Seleccion", "Club",
    "Jugador", "Entrenador", "Arbitro",
    "Estadio", "Ciudad", "Pais",
    "Grupo", "Fase", "Premio",
    "EventoGol", "EventoTarjeta", "EventoSustitucion"
]
node_labels = basic_node_labels
rct_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=700,
    separators=["\n\n", "\n", ".", " "]
)
text_splitter = LangChainTextSplitterAdapter(rct_splitter)

# 1. Build KG and Store in Neo4j Database
kg_builder_pdf = SimpleKGPipeline(
   llm=ex_llm,
   driver=neo4j_driver,
   text_splitter=FixedSizeSplitter(chunk_size=500, chunk_overlap=100),
   embedder=embedder,
   entities=node_labels,
   relations=rel_types,
   prompt_template=original_prompt,
   from_pdf=False,
   perform_entity_resolution=True
)

await kg_builder_pdf.run_async(text=text)


In [ ]:

from neo4j_graphrag.experimental.components.resolver import (
    #SinglePropertyExactMatchResolver,
     SpaCySemanticMatchResolver,
    # FuzzyMatchResolver,
)
#resolver = SinglePropertyExactMatchResolver(neo4j_driver)  # exact match resolver
resolver = SpaCySemanticMatchResolver(neo4j_driver)  # semantic match with spaCy
# resolver = FuzzyMatchResolver(driver)  # fuzzy match with RapidFuzz
res = await resolver.run()
print(res)


In [ ]:
from neo4j import GraphDatabase

#Merge nodes
with neo4j_driver.session() as session:
    session.run("""
    MATCH (n:Entity)
    WITH n.name AS name, collect(n) AS nodes
    WHERE size(nodes) > 1
    CALL apoc.refactor.mergeNodes(nodes, {properties:"combine"})
    YIELD node
    RETURN node
    """)

In [ ]:
## DEBEMOS CREAR UN VECTOR INDEX PARA PODER CONSULTAR
import neo4j
from neo4j_graphrag.indexes import create_vector_index

create_vector_index(neo4j_driver, name="text_embeddings", label="Chunk",embedding_property="embedding", dimensions=1536, similarity_fn="cosine")

In [ ]:
##
##   CONSULTA  VectorRetriever
##


import neo4j
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings as Embeddings
from neo4j_graphrag.retrievers import VectorRetriever,HybridRetriever

from neo4j_graphrag.generation.graphrag import GraphRAG
from neo4j_graphrag.llm import OpenAILLM as LLM
from langchain.embeddings.openai import OpenAIEmbeddings
import json
import os

pregunta = "Cual jugador del Cruzeiro le hizo gol a España"



# 2. KG Retriever
vector_retriever = VectorRetriever(neo4j_driver,index_name="text_embeddings",embedder=embedder)
vector_resp  = vector_retriever.get_search_results(query_text=pregunta, top_k=5)

print("VECTOR RETRIEVER")
for record  in vector_resp.records:
  print("===========\n" + json.dumps(record, indent=4))

print("====================================================")




In [ ]:
import neo4j
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings as Embeddings
from neo4j_graphrag.retrievers import VectorRetriever,HybridRetriever,VectorCypherRetriever

from neo4j_graphrag.generation.graphrag import GraphRAG
from neo4j_graphrag.llm import OpenAILLM as LLM
from langchain.embeddings.openai import OpenAIEmbeddings
import json
import os



retrieval_query = """MATCH (n)-[:FROM_CHUNK]->(node)
RETURN collect(n) AS sources
"""

retrieval_query = """


WITH node AS chunk

// Buscar nodos _Entity_ conectados a este chunk
OPTIONAL MATCH (e1:_Entity_)-[:FROM_CHUNK]->(chunk)

// Obtener relaciones entre entidades
OPTIONAL MATCH (e1)-[r]-(e2:_Entity_)
WHERE NOT type(r) IN ['FROM_CHUNK', 'FROM_DOCUMENT', 'NEXT_CHUNK']

// Obtener los chunks de origen de las entidades relacionadas
OPTIONAL MATCH (e2)-[:FROM_CHUNK]->(c2:Chunk)

RETURN
    chunk,
    collect(DISTINCT e1) AS entities_1,
    collect(DISTINCT e2) AS entities_2,
    collect(DISTINCT r) AS relationships,
    collect(DISTINCT c2) AS related_chunks


"""


retrieval_query = """
WITH node AS chunk

// Buscar nodos _Entity_ conectados a este chunk
OPTIONAL MATCH (e1:_Entity_)-[:FROM_CHUNK]->(chunk)

// Obtener relaciones entre entidades
OPTIONAL MATCH (e1)-[r]-(e2:_Entity_)
WHERE NOT type(r) IN ['FROM_CHUNK', 'FROM_DOCUMENT', 'NEXT_CHUNK']

// Obtener los chunks de origen de las entidades relacionadas
OPTIONAL MATCH (e2)-[:FROM_CHUNK]->(c2:Chunk)

// Retornar sin incluir la propiedad 'embedding'
RETURN
    chunk { .* , embedding: null } AS chunk,
    [e IN collect(DISTINCT e1) | e { .* , embedding: null }] AS entities_1,
    [e IN collect(DISTINCT e2) | e { .* , embedding: null }] AS entities_2,
    collect(DISTINCT r) AS relationships,
    [c IN collect(DISTINCT c2) | c { .* , embedding: null }] AS related_chunks


"""

retrieval_query = """
CALL db.index.vector.queryNodes('text_embeddings', $top_k, $query_vector)
YIELD node AS chunk, score AS similarity_score
MATCH (chunk)-[r]->(m)
RETURN chunk, r, m, similarity_score
LIMIT 5
"""

retrieval_query = "MATCH (node) RETURN node"

retrieval_query_c2 = """
CALL db.index.vector.queryNodes('text_embeddings', $top_k, $query_vector)
YIELD node AS chunk, score AS similarity_score
MATCH (chunk)-[r]->(child)-[rr]->(child2)
RETURN
  similarity_score,
  r,
  rr,
  apoc.map.removeKey(properties(chunk), 'embedding') AS chunk,
  apoc.map.removeKey(properties(child), 'embedding') AS child,
  apoc.map.removeKey(properties(child2), 'embedding') AS child2
ORDER BY similarity_score DESC
"""


retrieval_query_u = """
CALL db.index.vector.queryNodes('text_embeddings', $top_k, $query_vector)
YIELD node AS chunk, score AS similarity_score
MATCH (chunk)
RETURN DISTINCT
  similarity_score,
  apoc.map.removeKey(properties(chunk), 'embedding') AS chunk
UNION
CALL db.index.vector.queryNodes('text_embeddings', $top_k, $query_vector)
YIELD node AS chunk, score AS similarity_score
MATCH (chunk)-[r]->(child)-[rr]->(child2)
RETURN DISTINCT
  similarity_score,
  apoc.map.removeKey(properties(child), 'embedding') AS chunk
ORDER BY similarity_score DESC

"""

re_hops ="""
//1) Go out up to N hops in the entity graph and get relationships
WITH node AS chunk
MATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,4}()
UNWIND relList AS rel

//2) collect relationships and text chunks
WITH collect(DISTINCT chunk) AS chunks,
 collect(DISTINCT rel) AS rels

//3) format and return context
RETURN '=== text ===n' + apoc.text.join([c in chunks | c.text], 'n---n') + 'nn=== kg_rels ===n' +
 apoc.text.join([r in rels | startNode(r).name + ' - ' + type(r) + '(' + coalesce(r.details, '') + ')' +  ' -> ' + endNode(r).name ], 'n---n') AS info
"""


pregunta = "en que club juega el arquero a quien le sacaron tarjeta amarilla por juego peligroso"

vector_cypher_retriever = VectorCypherRetriever(neo4j_driver, "text_embeddings", re_hops, embedder)
vc_resp = vector_cypher_retriever.get_search_results(query_text=pregunta, top_k=5)

print("VectorCypherRetriever ",pregunta)
print(vc_resp)
print("------------")
for record  in vc_resp.records:
  print(type(record))
  print(record)
  #print("===========\n" + json.dumps(record, indent=4))

print("====================================================")




In [ ]:
llm = LLM(model_name="gpt-5-nano")
rag = GraphRAG(llm=llm, retriever=vector_cypher_retriever)

pregunta = "en que club juega el arquero a quien le sacaron tarjeta amarilla por juego peligroso o su nacionalidad"
print("PREGUNTA",pregunta)
resp = rag.search(pregunta)
print(resp)


In [ ]:
from neo4j_graphrag.generation import RagTemplate, GraphRAG

prompt_template = RagTemplate(
    template="Answer the question using only context {context} and examples",
    expected_inputs=["context"]
)

llm = LLM(model_name="gpt-4o-mini")
rag = GraphRAG(retriever=vector_cypher_retriever, llm=llm, prompt_template=prompt_template)

pregunta = "En cual equipo (club) juega quien  hizo gol el ultimo gol de la final"
print("PREGUNTA",pregunta)
resp = rag.search(pregunta)
print(resp)

In [ ]:
import re
import networkx as nx
import matplotlib.pyplot as plt


def parse_kg_edges_from_info(info: str):
    sep = "\n\n=== kg_rels ===\n"
    pos = info.find(sep)
    if pos == -1:
        return [], ""
    text_part = info[:pos]
    kg_part = info[pos + len(sep):]
    lines = [ln.strip() for ln in kg_part.split("\n") if ln.strip() and ln.strip() != '---']
    edges = []
    for ln in lines:
        # expected: <start> - <TYPE> -> <end>
        # split on ' - ' then ' -> '
        if ' - ' in ln and ' -> ' in ln:
            left, _, rest = ln.partition(' - ')
            type_, _, right = rest.partition(' -> ')
            start = left.strip()
            end = right.strip()
            edges.append((start, type_.strip(), end))
    return edges, text_part


def visualize_vc_subgraph(vc, query_text: str, top_k: int = 1):
    res = vc.get_search_results(query_text=query_text, top_k=top_k)
    info = res.records[0]['info']
    edges, text_part = parse_kg_edges_from_info(info)

    print("# Text Chunk Context (truncated):")
    print(text_part[:600])

    if not edges:
        print("\n(No KG relationships parsed. Try one-hop retriever or adjust query.)")
        return

    # Build graph
    G = nx.DiGraph()
    for s, t, e in edges:
        G.add_node(s)
        G.add_node(e)
        G.add_edge(s, e, label=t)

    # Draw
    pos = nx.spring_layout(G, seed=42, k=1.5, iterations=200)
    plt.figure(figsize=(14, 10))
    nx.draw(G, pos, with_labels=True, node_color='lightyellow', node_size=1800, edge_color='gray', font_size=10)
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=9)
    plt.title("VC Subgraph (parsed from kg_rels)", fontsize=14)
    plt.show()

simple_multihop = """
WITH node AS chunk
// Just get entities that have relationships (ignoring how they connect to chunk)
MATCH (e1)-[r]-(e2)
WHERE NOT (e1:Chunk OR e1:Document OR e1:_KGBuilder_)
  AND NOT (e2:Chunk OR e2:Document OR e2:_KGBuilder_)
  AND NOT type(r) IN ['NEXT_CHUNK', 'FROM_DOCUMENT', 'FROM_CHUNK']
WITH chunk, collect(DISTINCT r) AS rels LIMIT 1
RETURN '=== text ===\\n' + chunk.text + '\\n\\n=== kg_rels ===\\n' +
       apoc.text.join([r IN rels |
            coalesce(startNode(r).name, startNode(r).id, elementId(startNode(r))) +
            ' - ' + type(r) + ' -> ' +
            coalesce(endNode(r).name, endNode(r).id, elementId(endNode(r)))
       ], '\\n---\\n') AS info
"""

vc_simple = VectorCypherRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query=simple_multihop
)

try:
    visualize_vc_subgraph(vc_simple, query_text="seleccion colombia", top_k=1)
except Exception as e:
    print("Simple multi-hop error:", e)

In [ ]:


query = "Cuales son los jugadores con las goles y que para que equipos juegan los que mas los asisten"
vector_results = vector_retriever.get_search_results(query_text=query, top_k=3)
cypher_results = vector_cypher_retriever.get_search_results(query_text=query, top_k=3)

def clean_results(results):
    # Remove query_vector from results metadata
    if hasattr(results, 'metadata') and 'query_vector' in results.metadata:
        del results.metadata['query_vector']

    # Also clean individual records if needed
    if hasattr(results, 'records'):
        for record in results.records:
            # Check metadata
            if hasattr(record, 'metadata') and 'query_vector' in record.metadata:
                del record.metadata['query_vector']

    return results

vector_results = clean_results(vector_results)
cypher_results = clean_results(cypher_results)

print("VECTOR RESULTS:")
print(vector_results)
print("\nCYPHER RESULTS:")
print(cypher_results)

In [ ]:
def process_results(query, results):
    system_prompt = f"""Eres un experto en análisis de informacion deportiva.
Has recibido resultados de un sistema de recuperación de información.

Tu tarea es:
1. Analizar los resultados de recuperación proporcionados
2. Responder la pregunta del usuario de manera completa y precisa
3. Proporcionar información relevante basada en el contexto recuperado

Responde en español de manera clara y estructurada."""

    user_prompt = f"""
PREGUNTA DEL USUARIO: {query}

Retrieval Results: {str(results)}

Por favor, analiza los resultados de recuperación y responde la pregunta del usuario de manera completa.
"""

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt)
    ]

    try:
        response = gemini_model.invoke(messages)
        return response.content
    except Exception as e:
        return f"Error al procesar con Gemini: {e}"

print("\nRespuesta Con Vectores")
print("="*80)
response = process_results(query, vector_results)
print(response)
print("="*80)

print("\nRespuesta Con Vectores + Nodes")
print("="*80)
response = process_results(query, cypher_results)
print(response)
print("="*80)